In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = "C:/Users/szieg/Repositories/Credit_Risk_Analysis/Module-17-Challenge-Resources/LoanStats_2019Q1.csv"
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
#Convert String Values into Numerial values 

df = pd.get_dummies(df, columns=["home_ownership","verification_status", "pymnt_plan", "hardship_flag", "debt_settlement_flag"])
df.head(100)

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,Mar-2019,low_risk,27.24,0.0,0.0,8.0,...,0,0,0,1,0,1,0,1,1,1
1,25000.0,0.2000,929.09,105000.0,Mar-2019,low_risk,20.23,0.0,0.0,17.0,...,0,1,0,0,0,0,1,1,1,1
2,20000.0,0.2000,529.88,56000.0,Mar-2019,low_risk,24.26,0.0,0.0,8.0,...,0,1,0,0,0,0,1,1,1,1
3,10000.0,0.1640,353.55,92000.0,Mar-2019,low_risk,31.44,0.0,1.0,10.0,...,0,0,0,1,0,0,1,1,1,1
4,22000.0,0.1474,520.39,52000.0,Mar-2019,low_risk,18.76,0.0,1.0,14.0,...,0,1,0,0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20000.0,0.0881,634.23,115000.0,Mar-2019,low_risk,32.58,1.0,0.0,13.0,...,0,1,0,0,1,0,0,1,1,1
96,24000.0,0.1240,538.74,50000.0,Mar-2019,low_risk,25.88,0.0,1.0,12.0,...,0,0,0,1,0,1,0,1,1,1
97,14000.0,0.1390,325.04,60000.0,Mar-2019,low_risk,11.36,0.0,1.0,5.0,...,0,0,0,1,1,0,0,1,1,1
98,40000.0,0.0702,1235.45,150000.0,Mar-2019,low_risk,13.66,0.0,0.0,17.0,...,0,1,0,0,0,1,0,1,1,1


In [7]:
df.dtypes

loan_amnt                              float64
int_rate                               float64
installment                            float64
annual_inc                             float64
issue_d                                 object
                                        ...   
verification_status_Source Verified      uint8
verification_status_Verified             uint8
pymnt_plan_n                             uint8
hardship_flag_N                          uint8
debt_settlement_flag_N                   uint8
Length: 91, dtype: object

In [8]:
df['issue_month'] = pd.DatetimeIndex(df['issue_d']).month
df['issue_year'] = pd.DatetimeIndex(df['issue_d']).year
df = df.drop(["issue_d"], axis=1)
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,issue_month,issue_year
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,1,0,1,0,1,1,1,3,2019
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,0,0,0,1,1,1,1,3,2019
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,0,0,0,1,1,1,1,3,2019
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,1,0,0,1,1,1,1,3,2019
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,1,0,0,1,1,1,3,2019


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['loan_status'] = le.fit_transform(df['loan_status']) 
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,issue_month,issue_year
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,0,1,0,1,0,1,1,1,3,2019
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,0,0,0,0,1,1,1,1,3,2019
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,0,0,0,0,1,1,1,1,3,2019
3,10000.0,0.1640,353.55,92000.0,1,31.44,0.0,1.0,10.0,1.0,...,0,1,0,0,1,1,1,1,3,2019
4,22000.0,0.1474,520.39,52000.0,1,18.76,0.0,1.0,14.0,0.0,...,0,0,1,0,0,1,1,1,3,2019


# Split the Data into Training and Testing

In [10]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

# Create our target
y = df.loan_status

In [11]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,hardship_flag_N,debt_settlement_flag_N,issue_month,issue_year,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,3,2019,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,3,2019,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,3,2019,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,3,2019,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,3,2019,0,1,0,1,1,0


In [12]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: loan_status, dtype: int32

In [13]:
df.dtypes

loan_amnt                 float64
int_rate                  float64
installment               float64
annual_inc                float64
loan_status                 int32
                           ...   
pymnt_plan_n                uint8
hardship_flag_N             uint8
debt_settlement_flag_N      uint8
issue_month                 int64
issue_year                  int64
Length: 92, dtype: object

In [14]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,hardship_flag_N,debt_settlement_flag_N,issue_month,issue_year,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.0,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,1.0,1.726172,2019.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.0,0.743862,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,1.0,1.000000,2019.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,1.0,1.000000,2019.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,1.0,2.000000,2019.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,1.0,2.000000,2019.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.0,3.000000,2019.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [16]:
# Split into Train and Test Sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(51612, 94)
(17205, 94)
(51612,)
(17205,)


In [17]:
# Scale data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [18]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
brf_model_fit = brf_model.fit(X_train_scaled, y_train)


In [19]:
predictions = brf_model.predict(X_test_scaled)
predictions

array([1, 1, 1, ..., 1, 1, 0])

In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

bal_acc_score = balanced_accuracy_score(y_test, predictions)
bal_acc_score

0.7659675462401245

In [21]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df.head()

,Predicted High Risk,Predicted Low Risk
Actual High Risk,67,34
Actual Low Risk,2248,14856


In [22]:
# Print the imbalanced classification report
# Displaying results

print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score : {bal_acc_score}")
print()
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Confusion Matrix


,Predicted High Risk,Predicted Low Risk
Actual High Risk,67,34
Actual Low Risk,2248,14856


Balanced Accuracy Score : 0.7659675462401245

Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.66      0.87      0.06      0.76      0.56       101
          1       1.00      0.87      0.66      0.93      0.76      0.59     17104

avg / total       0.99      0.87      0.66      0.92      0.76      0.59     17205



In [23]:
# List the features sorted in descending order by feature importance
importance = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
importance

[(0.06976404371489758, 'total_pymnt'),
 (0.06012681105590557, 'last_pymnt_amnt'),
 (0.0594949979740323, 'total_rec_int'),
 (0.05138326326410982, 'total_rec_prncp'),
 (0.05126934450393019, 'total_pymnt_inv'),
 (0.03369524241154661, 'int_rate'),
 (0.029801936019043084, 'issue_month'),
 (0.018257312411745198, 'installment'),
 (0.017997871456793324, 'revol_bal'),
 (0.016634249991546668, 'out_prncp'),
 (0.016548527562581385, 'tot_hi_cred_lim'),
 (0.01651120171967778, 'out_prncp_inv'),
 (0.0160950120533944, 'dti'),
 (0.016034360211997875, 'annual_inc'),
 (0.015834290533723903, 'mths_since_recent_inq'),
 (0.01547048054538417, 'max_bal_bc'),
 (0.015468733475720968, 'bc_open_to_buy'),
 (0.015221148504891018, 'bc_util'),
 (0.015188316323335681, 'mo_sin_old_il_acct'),
 (0.014777946982263725, 'tot_cur_bal'),
 (0.014644658848639174, 'mo_sin_old_rev_tl_op'),
 (0.014335554612972206, 'total_rev_hi_lim'),
 (0.013619593143173044, 'all_util'),
 (0.013604990628736404, 'total_bal_ex_mort'),
 (0.01348272702

### Easy Ensemble AdaBoost Classifier

In [24]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

eec = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
eec.fit(X_train_scaled, y_train) 

y_pred = eec.predict(X_test_scaled)
y_pred


array([1, 1, 1, ..., 1, 1, 1])

In [25]:
# Calculated the balanced accuracy score
bal_acc_score = balanced_accuracy_score(y_test, y_pred)
bal_acc_score

0.9310754128499847

In [26]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df.head()

,Predicted 0,Predicted 1
Actual 0,67,34
Actual 1,2248,14856


In [27]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score : {bal_acc_score}")
print()
print("Imbalanced Classification Report")
print()
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,67,34
Actual 1,2248,14856


Balanced Accuracy Score : 0.9310754128499847

Imbalanced Classification Report

                   pre       rec       spe        f1       geo       iba       sup

          0       0.08      0.92      0.94      0.16      0.93      0.87       101
          1       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.96      0.93      0.87     17205

